In [ ]:
from google.cloud import bigquery
import pandas as pd

In [ ]:
client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

In [ ]:
client = bigquery.Client(location="US", project="ml-session")

# Send a query to a Public dataset

In [ ]:
query = """
SELECT
  vendor_id,
  passenger_count,
  trip_distance,
  rate_code,
  payment_type,
  total_amount,
  tip_amount
FROM
  `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2018`
WHERE tip_amount >= 0
LIMIT 100
"""
query_job = client.query(
    query,
     location="US",
)

df = query_job.to_dataframe()
df.head(5)

# https://console.cloud.google.com/marketplace/product/city-of-new-york/nyc-tlc-trips

# Create a model by logistic regression

In [ ]:
query = """
CREATE OR REPLACE MODEL session10.taxi_tip_classification_model_demo2
OPTIONS
 (model_type='logistic_reg',
  input_label_cols=['tip_bucket'],
  -- enable_global_explain=true
  model_registry='vertex_ai',
  vertex_ai_model_id='taxi_tip_classification_model_demo',
  vertex_ai_model_version_aliases=['logistic_reg', 'experimental']
) AS
SELECT
  vendor_id,
  passenger_count,
  trip_distance,
  rate_code,
  payment_type,
  total_amount,
  CASE
    WHEN tip_amount > total_amount*0.20 THEN '20% or more'
    WHEN tip_amount > total_amount*0.15 THEN '15% to 20%'
    WHEN tip_amount > total_amount*0.10 THEN '10% to 15%'
  ELSE '10% or less'
  END AS tip_bucket
FROM
  `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2018`
WHERE tip_amount >= 0
  -- AND pickup_datetime >= '2018-10-01' AND dropoff_datetime < '2018-10-02'
  AND pickup_datetime >= '2018-11-02' AND dropoff_datetime < '2018-11-03'
"""

query_job = client.query(
    query,
    location="US",
)


In [ ]:
!ls

In [ ]:
json_request = {
  "instances": [{
  "vendor_id": "0", 
  "passenger_count": 1, 
  "trip_distance": 5.85, 
  "rate_code": "0", 
  "payment_type": "0",
  "total_amount": 55.56
  }]
}

In [ ]:
!cat command.sh

In [ ]:
!bash ./command.sh